In [1]:
pip install konlpy


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install kiwipiepy

Note: you may need to restart the kernel to use updated packages.


In [1]:
from tqdm import tqdm

In [2]:
import sys
import os
import pandas as pd

sys.path.append('/app/EZPZ_Project/modules/torchmodules') # 토치 모델 뉴스 요약 및 감정평가 가져오기

In [3]:
import pymysql
import cryptography
import sql_connection as sc #mysql connection

In [4]:
from privates.ezpz_db import * #db연동 정보

In [5]:
from tokenizer import get_keyword_nnp, get_keyword_nng #토크나이저 호출 nnp는 

In [6]:
#토치 모듈 키워드추출 토크나이저와 t5 base sum 리뷰 요약
#import tokenizer 토크나이저는 일단 나중에
from service_models import ServiceModels #t5model

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
data_check= ServiceModels() #모델 서빙 모듈 객체

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


1. 분기별로 데이터를 나눠서 정보 가져오는 함수: data_split_by_quater() return 분기별 리뷰 리스트/ 03(1분기) 04(2분기) 05(3분기) 06(4분기)
2. 반기별로 데이터를 나눠서 정보 가져오는 함수: data_split_by_half() return 반기별 리뷰 리스트/ 01(전반기) 02(하반기)
3. 모든 기간 데이터로 정보 가져오는 함수 data_not_split 00: 모든 기간 return 모든기간 리뷰 리스트(해당 회사 전부 리뷰 요약)
4. 키워드 추출하는 함수 (데이터 나눠서 크롤링하는 함수에 넣어줌) get_keyword(review_list) 이지가 리스트로 받기도 가능하다 했으니
5. 리뷰 내용 요약 함수(매개변수: 분기/반기 별로 나눠진 리뷰 리스트) (크롤링하는 함수에 넣어줌)
6. 이모든걸 실행해주는 주요 함수: review_sum(all_comp_name):


In [10]:
def get_sum_uid(): # 필요할지 안필요할지 미지수 딕셔너리에 다 넣어줄수도 있음 그냥
    pass

In [94]:
def get_year_list(split_data): #분기/반기별 분할된 데이터를 그룹핑할 기준인 연도리스트 얻음
    i = 0
    year_list = [] #년도 개수 체크
    for item in range(len(split_data)):
        year = split_data[item][3][:4]  # YYYY 형식의 연도 추출
        if year not in year_list:
            year_list.append(year) #한 분기/반기에 포함돼있는 연도 리스트 [2023 ,1992...]
    return year_list #이걸로 그루핑 하고 uid 얻을때도 쓰고



In [79]:
tk= data_split_by_quater()
len(tk)

4

In [80]:
tk[0]

((1,
  '온보딩을 기초로 입사부터 체계적으로 회사에 적응할 수 있는 환경을 만들어 줍니다! 다른 회사에서 좋았던 복지가 하나로 뭉친 느낌으로 많은 혜택을 받고 있습니다 :D (그중 유연출퇴근제도 너무 좋아요) 자기계발 할 수 있는 기회가 많아 업무와 연계하여 공부하고 있습니다!',
  5,
  '202302'),
 (1,
  '똑똑한 팀원들, 수평적인 업무 분위기, 도움이 필요할 때 손을 내미면 다들 적극적으로 도와주심, 성장의 기회가 많음, 복지가 좋음, 분위기가 밝고 대체로 화기애애함. 의사결정이 투명하게 이루어짐.',
  5,
  '202301'),
 (1, '매출 규모가 작음. 다양한 거래처 및 영업 확장이 필요합니다. 그 외 딱히 단점은 없습니다.', 5, '202302'),
 (1,
  '딱히 떠오르는 것은 없음. 하나 꼽자면 재택근무가 불가능. 그런데 유연근무제나 자리비움이 가능해서 급한 일이 있을 때 스케줄을 조절해서 일할 수 있음.',
  5,
  '202301'),
 (1,
  '1인 1법인카드가 일단 최고의 장점 입니다. 구성원의 성장에 대한 지원을 아낌없이 해 줍니다. 피드백 세션을 통해서 부족한게 있는지, 현재 하는 업무에서의 어려운 점을 솔직하게 나눌 수 있는 자리 또한 좋습니다.',
  4,
  '202202'),
 (1,
  '젊은 회사라서 좋긴 하지만 시니어가 더 많이 필요하다는 것. 다양한 배경을 가진 사람들이 채용 되었으면 좋겠다. 은근히 장비 지원이 짠다.',
  4,
  '202202'),
 (1,
  '1. 편안한 분위기       - 자유로운 연차 사용       - 자유로운 출퇴근 (11-4만 지키면 자유롭게 출퇴근 가능)       - 휴식 쓰고 싶을 때 쓸 수 있음    2. 복지       - 개인 법카를 줌,,, 먹고 싶은 음식 자유롭게 먹을 수 있다는 거(제일 좋음)       - 업무 외 자기계발비 지원해줘서 좋음       - 휴가비 40만원 지원       - 이외에도 규모에 비해 복지가 많다고 생

In [ ]:
def no_data_split(): #스플릿안된 데이터 넘겨주는 함수
    pass

In [ ]:
def data_split_by_half():# 반기별로 분활하는 함수(밑에 sql문 참고)
    pass

In [78]:

def data_split_by_quater(): #회사이름과 시간분할 기준 이제 회사별로 이걸 해줘야지

        quater_data = [] #분기들을 모아놓은 리스트
        sql = ' SELECT comp_uid, review_cont, review_rate, review_date ' #senti는 나중에  pred한걸로 수정 요약내용,감성평가,리뷰평점,리뷰날짜
        sql += " FROM comp_review "
        sql += f" WHERE review_date LIKE '____01' OR review_date LIKE '____02' OR review_date LIKE '____03'; " #1분기의 리뷰 데이터

        #1분기 데이터 연도 구분 x
        quater_one_data = sc.conn_and_exec(sql)



        sql = ' SELECT comp_uid,review_cont, review_rate, review_date ' #senti는 나중에  pred한걸로 수정 요약내용,감성평가,리뷰평점,리뷰날짜
        sql += " FROM comp_review "
        sql += " WHERE review_date LIKE '____04' OR review_date LIKE '____05' OR review_date LIKE '____06'; "

        #2분기 데이터 연도 구분 x
        quater_two_data = sc.conn_and_exec(sql)

        sql = ' SELECT comp_uid, review_cont, review_rate, review_date ' #senti는 나중에  pred한걸로 수정 요약내용,감성평가,리뷰평점,리뷰날짜
        sql += " FROM comp_review "
        sql += " WHERE review_date LIKE '____07' OR review_date LIKE '____08' OR review_date LIKE '____09'; "

        #3분기 데이터 연도 구분 x
        quater_three_data = sc.conn_and_exec(sql)

        sql = ' SELECT comp_uid, review_cont, review_rate, review_date ' #senti는 나중에  pred한걸로 수정 요약내용,감성평가,리뷰평점,리뷰날짜
        sql += " FROM comp_review "
        sql += " WHERE review_date LIKE '____10' OR review_date LIKE '____11' OR review_date LIKE '____12'; "

        #4분기 데이터 연도 구분 x
        quater_four_data = sc.conn_and_exec(sql)

        quater_data.append(quater_one_data)
        quater_data.append(quater_two_data)
        quater_data.append(quater_three_data)
        quater_data.append(quater_four_data)
        return quater_data

    #data[0] 리뷰내용/ data[1] 리뷰감성평가/ data[2] 리뷰평점/ data[3] 리뷰날짜









In [100]:
crawling_all

[[[1, '온보딩을 기초로 회사에 적응할 수 있는 환경을 만들어 주는 유연출퇴근제도는 업무와 연계하여 공부하고 있다.']],
 [[1, '1인 1법인카드는 구성원의 성장에 대한 지원을 해주고 피드백 세션을 통해 부족한 점을 솔직하게 나눌 수 있는 자리가 좋다.'],
  [2, '회사 출근하는 것이 즐거운 적은 처음이라 회사 출근하는 것이 즐거운 적은 처음이다.'],
  [3, 'okr을 통해 성과보상이 높음 복지가 높고 매출에 집중을 많이 하고 있다.']],
 [[3,
   '회사가 성장이 빠른 탓에 체계가 부족한 부분들이 있으나 매월 계속 개선해나가는 것을 바로 옆에서 볼수 있고 직원들 간에 사이가 좋아 회사가 빠르게 커진다는 직접 경험이 가능하다.']],
 [[4, '3년 정도 꾸준히 배워야 경력기회를 얻을 수 있다.']],
 [[4, '능력에 따라 승진과 급여 인상이 빠르게 이루어진다.']],
 [[4,
   '사무실 환경 좋고 사내문화와 굉장히 좋은 편임 컨설팅 품질 및 기업 이미지 관리에 매우 민감함 인턴 지원의 경우 타 컨설팅펌보다 업무강도가 높은 것으로 알고 있음 글로벌펌에 비해 급여에서 불만족할 수 있음 글로벌펌에 비해 급여에서 불만족할 수 있음 급여에서 불만족할 수 있음 업무강도가 높은 것으로 알고 있음 등의 철학을 가장 중요시']],
 [[4,
   '초봉은 전략 Firm에 비해 낮을 수 있지만 본인의 성과에 따라 단기간에 sump 가능 유연한 업무 환경 및 전문가에 대한 존중과 육성 등 다양한 산업과 중요한 의사결정 과정에 대해 배울 수 있음 Quality를 위해서는 모든 노력을 다 기울이는 회사 인사조직 컨설팅 일 배우기에는 최적의 장소 업계를 리드했던 전문가들에게 업무와 전문지식']],
 [],
 []]

In [28]:
def get_keyword_list(comp_uid_review_sum): #연도 및 분/반기 요약돼있는 dict 받아서 키워드 분할 counter도 써서 빈도수 나오면(1)을 옆에 _빈도수 써줘서 for문돌려서 #추가 하나키워드 나올때마다
     #keyword_hashtag_list = []
     #for key,value in sum_dict.items():
     comp_review_list=[]
     year_keyword_list = []
     keyword_hashtag_list = []
     for uid_reviews in comp_uid_review_sum: #각연도의 uid와 uid별 review들을 묶어둔 이차원 리스트
          for comp_uid_review in uid_reviews: # uid 하나와 리뷰로 이루어진 1차원리스트
               comp_review_list.append(comp_uid_review[1]) #[1]이 각회사 요약 리뷰
               #리스트로 바꾸고 다시 인덱싱으로 원소 꺼내와야될듯
               review_keyword = get_keyword_nng([comp_uid_review[1]],'review') #[1]이 각회사의 요약된 리뷰
               #print(review_keyword) #각 회사별 연도,분.반기 구분된 요약 내용의 키워드 추출 여기서 키워드 추출해서 회사별로 #과 _빈도수 붙여주기
               #연도별로 회사마다 나온 키워드를 저장
               year_keyword_list.append(review_keyword)

     for comp_keyword in year_keyword_list: #comp_keyword 는 한회사의 키워드로 ('온보딩',1) 꼴로 투플을 순회
          keyword_hashtag = '' # 키워드를 해시태그로 저장할 문장
          for keyword in comp_keyword:
               keyword_hashtag += keyword[0]+'_'+ str(keyword[1])+'#'
          keyword_hashtag =keyword_hashtag[:-1] #마지막 #은 제거
          keyword_hashtag_list.append(keyword_hashtag)


     return keyword_hashtag_list
     return year_keyword_list

          #temp_list.append(split_review)
          #return temp_list

In [44]:
get_keyword_list(crawling_all)

[]

In [ ]:
#quater_one_data[3]이 별점 회사별 한사람이 매긴 별점 comp_uid_review_sum이 

In [29]:
split_data = data_split_by_quater()

In [101]:
print(split_data[:10])

((1, '온보딩을 기초로 입사부터 체계적으로 회사에 적응할 수 있는 환경을 만들어 줍니다! 다른 회사에서 좋았던 복지가 하나로 뭉친 느낌으로 많은 혜택을 받고 있습니다 :D (그중 유연출퇴근제도 너무 좋아요) 자기계발 할 수 있는 기회가 많아 업무와 연계하여 공부하고 있습니다!', 'P', 5, '202302'), (1, '똑똑한 팀원들, 수평적인 업무 분위기, 도움이 필요할 때 손을 내미면 다들 적극적으로 도와주심, 성장의 기회가 많음, 복지가 좋음, 분위기가 밝고 대체로 화기애애함. 의사결정이 투명하게 이루어짐.', 'P', 5, '202301'), (1, '매출 규모가 작음. 다양한 거래처 및 영업 확장이 필요합니다. 그 외 딱히 단점은 없습니다.', 'N', 5, '202302'), (1, '딱히 떠오르는 것은 없음. 하나 꼽자면 재택근무가 불가능. 그런데 유연근무제나 자리비움이 가능해서 급한 일이 있을 때 스케줄을 조절해서 일할 수 있음.', 'N', 5, '202301'), (1, '1인 1법인카드가 일단 최고의 장점 입니다. 구성원의 성장에 대한 지원을 아낌없이 해 줍니다. 피드백 세션을 통해서 부족한게 있는지, 현재 하는 업무에서의 어려운 점을 솔직하게 나눌 수 있는 자리 또한 좋습니다.', 'P', 4, '202202'), (1, '젊은 회사라서 좋긴 하지만 시니어가 더 많이 필요하다는 것. 다양한 배경을 가진 사람들이 채용 되었으면 좋겠다. 은근히 장비 지원이 짠다.', 'N', 4, '202202'), (1, '1. 편안한 분위기       - 자유로운 연차 사용       - 자유로운 출퇴근 (11-4만 지키면 자유롭게 출퇴근 가능)       - 휴식 쓰고 싶을 때 쓸 수 있음    2. 복지       - 개인 법카를 줌,,, 먹고 싶은 음식 자유롭게 먹을 수 있다는 거(제일 좋음)       - 업무 외 자기계발비 지원해줘서 좋음       - 휴가비 40만원 지원       - 이외에도 규모에 비해 복지가 많다고 생각해옴', 'P

In [99]:
#quater_one_data에서 요약한 애들 리스트를 합치기전에 받아서 코딩을 해야겠네 그러려면 sum_review_dict에서 일부코드만 따와야겠네
def get_avg_rate_list(split_data,year_list): #comp_uid_review_sum은 안쓴 확률 높음 평점항목이 없어서
    year_rate_average_list = [] #별점을 다 모아오는 리스트
    rate_sum = 0 #각회사의 연도 분/반기별 평점 합계 담아줄 변수 나중에 개수로 나눠 평균 구해줄 예정
    rate_count = 0 # 각 회사의 평점  개수
    uid_rate_year_list = []
    for year in year_list: #2023
        for item in range(len(split_data)): #테이블에서 가져온 데이터중 분기할 데이터들만 가져오기
                if split_data[item][3][:4] == year:
                    #split_data_dict[year].append(split_data[item][0]) #comp_uid를 딕셔너리에 넣어줌

                    uid_rate_year_list.append([year,split_data[item][0],split_data[item][2]]) #연도별(year) 회사uid와 리뷰 평점

                    """    if item+1 < len(split_data) and split_data[item][0] == split_data[item+1][0]: #순서대로 리뷰가 정렬되므로 이렇게 평점 더해줘도 됨 스플릿데이터 comp도 순서대로 1,2~
                             rate_sum += split_data[item][3] #그 회사의 평점 합을 더해줌
                             rate_count += 1 # 평점 개수 +1

                        else: #comp_uid 가 달라진경우
                              rate_sum+= split_data[item][0] #마지막으로 같았던 comp_uid의 평점 넣어줌
                              rate_count +=1
                              print(split_data[item][0])
                              year_rate_average_list.append([split_data[item][0],year,rate_sum/rate_count])"""

    #각 회사별 연도별 분/반기별 평점 리스트 갖고 오는 알고리즘

    result_dict = {}

    for item in uid_rate_year_list:
        year = item[0]
        key = (year, item[1]) # (연도, 두 번째 인덱스 값)을 키로 사용

        if key in result_dict: #딕셔너리안에 키가 없으면 항목 넣어주고 키에 맞춰서 append 평점 
            result_dict[key].append(item[2])
        else:
            result_dict[key] = [item[2]]


    for key in result_dict:
        values = result_dict[key]
        average_value = sum(values) / len(values)
        year_rate_average_list.append([key[0], key[1], round(average_value,1)])
    return year_rate_average_list#year_rate_average_list


In [81]:
#1. 연도별로 seperate 2. 연도별 seperate 된 애들중 같은 분기끼리는 전부 concat 요약 모델 for문
def sum_review_dict(split_data,year_list,split_time): #분기/반기별로 모아놓은 list quater_data[0]이 1분기 [1] [2] [3] 이 각각 분기 [0][0]은 그 분기정보중 한 row [0][0][0]은cont 하나
    split_data_dict ={}
    new_value_list = []
    #combined_sum_review_list = []
    i = 0 #얼마나 왔는지 체크용
    for year in year_list:
        split_data_dict[year] = [] #연도별로

    for year in year_list:
        for item in range(len(split_data)):
            if split_data[item][3][:4] == year:
                #split_data_dict[year].append(split_data[item][0]) #comp_uid를 딕셔너리에 넣어줌
                split_data_dict[year].append([split_data[item][0],split_data[item][1]]) #분기/반기 리뷰를 또 연도별로 구분해서 담아줌 uid를 어떻게 구분해서 join하지
                #일단 현재 튜플로 comp_uid와 리뷰내용을 묶어놓은 상태 여기서 어떻게 분리해서 요약따로하지

    #이제 이 내용 담긴 딕셔너리를 key별(연도)로 concat 하면 dict안에 연도별 분기별로 구분된 요약내용이 담긴 딕셔너리 완성
    for key,value in split_data_dict.items(): #딕셔너리 안에 있는 연도별 value들
        #for tp in value: #각 연도별 리스트 안에 있는 튜플들(리뷰와 comp_uid)
        combined_sum_review_list = [] # 매 키가 순회할때마다 리스트 비워줌
        merged_reviews = []
        current_key = None
        current_text = ""

        for key, text in value:
            if key == current_key:
                current_text += " " + text
            else:
                if current_key is not None:
                    merged_reviews.append([current_key, current_text])
                current_key = key
                current_text = text

        # 마지막 데이터 처리
        if current_key is not None:
            merged_reviews.append([current_key, current_text]) # 각 연도별 리뷰같은회사끼리 한꺼번에 묶어준 튜플

        value =merged_reviews #각 연도별 같은 회사 묶어준 튜플

        for combined_review_index in tqdm(value): #각 tp가 생각해보니까 곧 리뷰 합친거잖아
            #combined_review = ''.join(tp[1]) #연도별 리뷰들을 한꺼번에 묶어줌
            combined_sum_review = data_check.get_summary(combined_review_index[1],'reviews_long') #연도별 분/반기별 묶인 리뷰 요약
            combined_sum_review_list.append(combined_sum_review)
            if combined_review_index[0]>5: #테스트용 코드
                break #테스트용으로 회사 100개만 확인
        print(i)
        i+=1

        new_value_temp = [[v[0], combined_sum_review_list[i]] for i, v in enumerate(value) if v[0]<5 ] #회사별 요약해준 값과 uid를 리스트로 묶어 다시 value 리스트에 넣어줌
        new_value_list.append(new_value_temp) #각 연도별 회사들의 리뷰 요약 을 리스트의 인덱스로 가지는 리스트 [[(1,~),(2,~)],[]]
        print(new_value_list)
    return new_value_list #일단은 comp_uid와 그에 맞는 요약된 리뷰를 담아주기 나중에 이 리스트들 한번에 합쳐주기
    
        #이것도 테스트용이라 value가 t[0]<10 까지만 해놨어 나중에 저 if절 빼면됨
    #for key, value in split_data_dict.items():
    #    split_data_dict[key] = new_value_list.pop(0)# 딕셔너리 키들에 한번에 묶어서 요약해준 문자열을 다시 넣어줌


    """for key,value in split_data_dict.items():
        if len(new_value_list) == 0: # 원래는 이코드 넣을 필요없는데 테스트코드라 딕셔너리 year에 포함안된회사(10개라) 있어서 길이가 달라짐 그러므로 테스트용
            break
        split_data_dict[key] = new_value_list.pop(0)# 딕셔너리 키들에 한번에 묶어서 요약해준 문자열을 다시 넣어줌

    return split_data_dict # 성공!!! 연도별 분/반기별 회사별 요약본 만들기 이제는 분/반기 그루핑하면 됨"""



In [33]:
def db_save():
    pass#get_all_comp_name

In [100]:
#함수들 다 실행해서 db까지 저장해주는 main function
"""
실행 메커니즘
1.리뷰 긁어올 회사를 받아옴
2.분기별/반기별 데이터를 분할
3.각 분할된 데이터에 있는 연도들을 리스트로 뽑아옴
4.그 연도와 분/반기를 보내 내용 요약 딕셔너리로 만들어줌
5.회사 아이디에 이어붙여
"""

def exec_all():

    ###
    #각 분기별 필요 데이터들 수집
    ###

    #1분기
    quater_data = data_split_by_quater() #분기별 데이터 분할 1,2,3,4 #나중에는 그냥 분기 4개 for문에 다넣어버려서 코드 길이 줄이자

    year_list_one = get_year_list(quater_data[0]) #1분기 "03"
    comp_uid_review_sum_one = sum_review_dict(quater_data[0],year_list_one,'03') #연도별 1분기 dict 매개변수 하나 추가해서 전반기 매개변수 넣어서 붙여주기 comp_uid도 나중에 키에 붙여 한번에하자
    keyword_list_one = get_keyword_list(comp_uid_review_sum_one) #리스트에 있는 모든 애들 키워드 가져와서 다시 value 튜플에 각각 넣어주기

    avg_rate_list_one = get_avg_rate_list(quater_data[0],year_list_one) #얘만 지금 한분기 전체 데이터가 다나옴 나머지는 잘랐어서 실제 실행할때는 테스트용 코드 전부 제거

    """#2분기
    year_list_two = get_year_list(quater_data[1]) #1분기 "03"
    comp_uid_review_sum_two = sum_review_dict(quater_data[1],year_list_one,'04') #연도별 1분기 dict 매개변수 하나 추가해서 전반기 매개변수 넣어서 붙여주기 comp_uid도 나중에 키에 붙여 한번에하자
    keyword_list_one = get_keyword_list(comp_uid_review_sum_two) #리스트에 있는 모든 애들 키워드 가져와서 다시 value 튜플에 각각 넣어주기

    avg_rate_list_two = get_avg_rate_list(quater_data[0],year_list_one)

    #3분기
    year_list_three = get_year_list(quater_data[2]) #1분기 "03"
    comp_uid_review_sum_three = sum_review_dict(quater_data[2],year_list_one,'05') #연도별 1분기 dict 매개변수 하나 추가해서 전반기 매개변수 넣어서 붙여주기 comp_uid도 나중에 키에 붙여 한번에하자
    keyword_list_three = get_keyword_list(comp_uid_review_sum_two) #리스트에 있는 모든 애들 키워드 가져와서 다시 value 튜플에 각각 넣어주기

    avg_rate_list_three = get_avg_rate_list(quater_data[2],year_list_one)
    #4분기
    year_list_four = get_year_list(quater_data[3]) #1분기 "03"
    comp_uid_review_sum_four = sum_review_dict(quater_data[3],year_list_one,'06') #연도별 1분기 dict 매개변수 하나 추가해서 전반기 매개변수 넣어서 붙여주기 comp_uid도 나중에 키에 붙여 한번에하자
    keyword_list_three = get_keyword_list(comp_uid_review_sum_two) #리스트에 있는 모든 애들 키워드 가져와서 다시 value 튜플에 각각 넣어주기

    avg_rate_list_four = get_avg_rate_list(quater_data[3],year_list_one)
"""


    #분기 딕셔너리에
    return comp_uid_review_sum_one,keyword_list_one,avg_rate_list_one
    #return(avg_rate_list)
    return year_list

    #quater_two_data = data_split_by_quater(all_comp_name,2) #분기별 데이터 분할 1,2,3,4
    #quater_three_data = data_split_by_quater(all_comp_name,3) #분기별 데이터 분할 1,2,3,4
    #quater_four_data = data_split_by_quater(all_comp_name,4) #분기별 데이터 분할 1,2,3,4



    #sum_review(quater_one_data,quater_two_data,quater_three_data,quater_four_data) #분기,반기 일단 분기만 했는데 내일 요거 토대로 반기도 넣어야됨

    #db_save(sum_review) #각각의 얻은 디비에 넣을 값들 바탕으로 매개변수 받아서 db세이브


In [101]:
crawling_all =  exec_all()

  0%|          | 2/856 [00:13<1:37:56,  6.88s/it]


0
[[[1, '온보딩을 기초로 회사에 적응할 수 있는 환경을 만들어 주고 자기계발 할 수 있는 기회가 많아 업무와 연계하여 공부하고 있다.']]]


  0%|          | 4/873 [00:23<1:23:25,  5.76s/it]


1
[[[1, '온보딩을 기초로 회사에 적응할 수 있는 환경을 만들어 주고 자기계발 할 수 있는 기회가 많아 업무와 연계하여 공부하고 있다.']], [[1, '1인 1법인카드는 구성원의 성장에 대한 지원을 해주고 피드백 세션을 통해 부족한 점을 솔직하게 나눌 수 있는 자리도 좋다.'], [2, '회사 출근하는 것은 최고의 기업문화를 갖고 있다.'], [3, 'odr을 통해 성과보상이 높음 복지가 높고 매출에 집중을 많이 하고 있다.']]]


  0%|          | 2/752 [00:14<1:29:37,  7.17s/it]


...

6
[[[1, '온보딩을 기초로 회사에 적응할 수 있는 환경을 만들어 주고 자기계발 할 수 있는 기회가 많아 업무와 연계하여 공부하고 있다.']], [[1, '1인 1법인카드는 구성원의 성장에 대한 지원을 해주고 피드백 세션을 통해 부족한 점을 솔직하게 나눌 수 있는 자리도 좋다.'], [2, '회사 출근하는 것은 최고의 기업문화를 갖고 있다.'], [3, 'odr을 통해 성과보상이 높음 복지가 높고 매출에 집중을 많이 하고 있다.']], [[3, '직원들 사이가 좋아 회사가 성장이 빠른 탓에 체계가 부족한 부분들이 있으나 매월 계속 개선해나가는 것을 바로 옆에서 볼수 있고 직원들 간에 사이가 좋아 회사가 빠르게 커진다는 직접 경험할 수 있다.']], [[4, '3년 정도 꾸준히 하면 경력기회를 얻을 수 있다.']], [[4, '능력에 따라 승진과 급여 인상이 빠르게 이루어진다.']], [[4, '사무실 환경 좋고 사내문화과 좋은 편이며 경영 컨설팅에 있어서 고객에게 가치를 제공하는 본질적인 철학을 가장 중요시한다.']], [[4, '단기간 집중적으로 성장할 수 있는 곳 초봉은 전략 Firm에 비해 낮을 수 있지만 본인의 성과에 따라 단기간에 jump 가능 유연한 업무 환경 및 전문가에 대한 존중과 육성 등 다양한 산업과 중요한 의사결정 과정에 대해 배울 수 있음 Quality를 위해서는 모든 노력을 다 기울이는 회사 인사조직 컨설팅 일 배우기에는 최적의 장소']]]


  0%|          | 1/892 [00:10<2:41:59, 10.91s/it]


7
[[[1, '온보딩을 기초로 회사에 적응할 수 있는 환경을 만들어 주고 자기계발 할 수 있는 기회가 많아 업무와 연계하여 공부하고 있다.']], [[1, '1인 1법인카드는 구성원의 성장에 대한 지원을 해주고 피드백 세션을 통해 부족한 점을 솔직하게 나눌 수 있는 자리도 좋다.'], [2, '회사 출근하는 것은 최고의 기업문화를 갖고 있다.'], [3, 'odr을 통해 성과보상이 높음 복지가 높고 매출에 집중을 많이 하고 있다.']], [[3, '직원들 사이가 좋아 회사가 성장이 빠른 탓에 체계가 부족한 부분들이 있으나 매월 계속 개선해나가는 것을 바로 옆에서 볼수 있고 직원들 간에 사이가 좋아 회사가 빠르게 커진다는 직접 경험할 수 있다.']], [[4, '3년 정도 꾸준히 하면 경력기회를 얻을 수 있다.']], [[4, '능력에 따라 승진과 급여 인상이 빠르게 이루어진다.']], [[4, '사무실 환경 좋고 사내문화과 좋은 편이며 경영 컨설팅에 있어서 고객에게 가치를 제공하는 본질적인 철학을 가장 중요시한다.']], [[4, '단기간 집중적으로 성장할 수 있는 곳 초봉은 전략 Firm에 비해 낮을 수 있지만 본인의 성과에 따라 단기간에 jump 가능 유연한 업무 환경 및 전문가에 대한 존중과 육성 등 다양한 산업과 중요한 의사결정 과정에 대해 배울 수 있음 Quality를 위해서는 모든 노력을 다 기울이는 회사 인사조직 컨설팅 일 배우기에는 최적의 장소']], []]


  0%|          | 0/520 [00:03<?, ?it/s]


8
[[[1, '온보딩을 기초로 회사에 적응할 수 있는 환경을 만들어 주고 자기계발 할 수 있는 기회가 많아 업무와 연계하여 공부하고 있다.']], [[1, '1인 1법인카드는 구성원의 성장에 대한 지원을 해주고 피드백 세션을 통해 부족한 점을 솔직하게 나눌 수 있는 자리도 좋다.'], [2, '회사 출근하는 것은 최고의 기업문화를 갖고 있다.'], [3, 'odr을 통해 성과보상이 높음 복지가 높고 매출에 집중을 많이 하고 있다.']], [[3, '직원들 사이가 좋아 회사가 성장이 빠른 탓에 체계가 부족한 부분들이 있으나 매월 계속 개선해나가는 것을 바로 옆에서 볼수 있고 직원들 간에 사이가 좋아 회사가 빠르게 커진다는 직접 경험할 수 있다.']], [[4, '3년 정도 꾸준히 하면 경력기회를 얻을 수 있다.']], [[4, '능력에 따라 승진과 급여 인상이 빠르게 이루어진다.']], [[4, '사무실 환경 좋고 사내문화과 좋은 편이며 경영 컨설팅에 있어서 고객에게 가치를 제공하는 본질적인 철학을 가장 중요시한다.']], [[4, '단기간 집중적으로 성장할 수 있는 곳 초봉은 전략 Firm에 비해 낮을 수 있지만 본인의 성과에 따라 단기간에 jump 가능 유연한 업무 환경 및 전문가에 대한 존중과 육성 등 다양한 산업과 중요한 의사결정 과정에 대해 배울 수 있음 Quality를 위해서는 모든 노력을 다 기울이는 회사 인사조직 컨설팅 일 배우기에는 최적의 장소']], [], []]


In [102]:
crawling_all

([[[1,
    '온보딩을 기초로 회사에 적응할 수 있는 환경을 만들어 주고 자기계발 할 수 있는 기회가 많아 업무와 연계하여 공부하고 있다.']],
  [[1, '1인 1법인카드는 구성원의 성장에 대한 지원을 해주고 피드백 세션을 통해 부족한 점을 솔직하게 나눌 수 있는 자리도 좋다.'],
   [2, '회사 출근하는 것은 최고의 기업문화를 갖고 있다.'],
   [3, 'odr을 통해 성과보상이 높음 복지가 높고 매출에 집중을 많이 하고 있다.']],
  [[3,
    '직원들 사이가 좋아 회사가 성장이 빠른 탓에 체계가 부족한 부분들이 있으나 매월 계속 개선해나가는 것을 바로 옆에서 볼수 있고 직원들 간에 사이가 좋아 회사가 빠르게 커진다는 직접 경험할 수 있다.']],
  [[4, '3년 정도 꾸준히 하면 경력기회를 얻을 수 있다.']],
  [[4, '능력에 따라 승진과 급여 인상이 빠르게 이루어진다.']],
  [[4, '사무실 환경 좋고 사내문화과 좋은 편이며 경영 컨설팅에 있어서 고객에게 가치를 제공하는 본질적인 철학을 가장 중요시한다.']],
  [[4,
    '단기간 집중적으로 성장할 수 있는 곳 초봉은 전략 Firm에 비해 낮을 수 있지만 본인의 성과에 따라 단기간에 jump 가능 유연한 업무 환경 및 전문가에 대한 존중과 육성 등 다양한 산업과 중요한 의사결정 과정에 대해 배울 수 있음 Quality를 위해서는 모든 노력을 다 기울이는 회사 인사조직 컨설팅 일 배우기에는 최적의 장소']],
  [],
  []],
 ['온보딩_1#기초_1#회사_1#적응_1#환경_1#자기_1#계발_1#기회_1#업무_1#연계_1',
  '카드_1#구성원_1#성장_1#지원_1#피드백_1#세션_1#부족_1#자리_1',
  '회사_1#출근_1#최고_1#기업_1#문화_1',
  '보상_1#복지_1#매출_1#집중_1',
  '직원_2#사이_2#회사_2#성장_1#체계_1#부족_1#부분_1#개선_1#경험_1',
  '정도_1#경력_1#기회_1',
  '능력_1#승진

In [96]:
for i in crawling_all:
    if i[1] == '2023':
        if i[0] == 18:
            print(1)




1


In [77]:
year_split_review_dict #이런식으로 만들어야됨

{'2023': [[1, '온보딩을 기초로 회사에 적응할 수 있는 환경을 만들어 주는 것이 장점이다.']],
 '2022': [[1,
   '1인 1법인카드는 구성원의 성장에 대한 지원을 해주고 피드백 세션을 통해 부족한 점을 솔직하게 나눌 수 있는 자리가 좋다.'],
  [2, '개인적으로 함께 일하는 직원들이 가장 중요한 요소 중 하나라고 생각하는데 회사 출근하는게 이렇게 즐거운 적은 처음입니다.'],
  [3,
   '복사는 복지가 높고 매출에 집중을 많이 하고 있고 재택근무 시스템이 잘 되어있어 주 1회출근만 하면 됨. OKR이 빡쳤고 진급심사가 있다.']],
 '2020': [[3, '회사가 성장이 빠른 탓에 체계가 부족한 부분들이 있으나 매월 계속 개선해나가는 것을 바로 옆에서 볼수 있다.']],
 '2019': [[4,
   '컨설팅 회사가 다 그렇듯이 클라이언트에게 보고할 시점이 되면 업무량이 많아지며 퇴근이나 주말 근무를 해야 할 때가 있다.']],
 '2018': [[4, '승진과 급여 인상이 빠르게 이루어진다.']],
 '2016': [[4, '업무강도가 높은 것으로 알고 있음.']],
 '2015': [[4,
   '초봉은 전략 Firm에 비해 낮을 수 있지만 본인의 성과에 따라 단기간에 jump 가능 유연한 업무 환경 및 전문가에 대한 존중과 육성 다양한 산업과 중요한 의사결정 과정에 대해 배울 수 있음 Quality를 위해서는 모든 노력을 다 기울이는 회사 인사조직 컨설팅 일 배우기에는 최적의 장소 업계를 리드했던 전문가들에게 업무와 전문지식을 배울 수']],
 '2021': [],
 '2017': []}

In [25]:
combined_data = []
current_key = None
current_text = ""

for review_list in year_split_review_dict.values():
    for review_tuple in review_list:
        key, text = review_tuple
        if key == current_key:
            current_text += " " + text
        else:
            if current_key is not None:
                combined_data.append((current_key, current_text))
            current_key = key
            current_text = text

    # 마지막 데이터 처리
    if current_key is not None:
        combined_data.append((current_key, current_text))

In [31]:
merged_tuples = []
current_key = None
current_text = ""

for key, text in year_split_review_dict['2023']:
    if key == current_key:
        current_text += " " + text
    else:
        if current_key is not None:
            merged_tuples.append((current_key, current_text))
        current_key = key
        current_text = text

# 마지막 데이터 처리
if current_key is not None:
    merged_tuples.append((current_key, current_text))

In [34]:
year_split_review_dict['2023'] =merged_tuples

In [ ]:
for tp in year_split_review_dict['2023']:
    print(tp[0])